In [67]:
# importing all required packages & notebook extensions at the start of the notebook
import pandas as pd
import biom
from scipy.stats import spearmanr               
from statsmodels.stats.multitest import multipletests 

%matplotlib inline

In [2]:
#needed variables
Data_raw='Data/raw'
Data_classified='Data/classified'
Data_aroma='Data/aroma'

<div style="background-color: skyblue; padding: 10px;">
    Titles
    </div>
<div style="background-color: aliceblue; padding: 10px;">
    Results

# Prepare Data
<div style="background-color: skyblue; padding: 10px;">


Export all necessary files

In [3]:
!qiime tools export \
  --input-path $Data_classified/taxonomy.qza \
  --output-path $Data_aroma

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported Data/classified/taxonomy.qza as TSVTaxonomyDirectoryFormat to directory Data/aroma


In [4]:
taxonomy = pd.read_csv("Data/aroma/taxonomy.tsv", sep="\t")

In [5]:
!qiime tools export \
  --input-path $Data_classified/table-filtered-sourdough_only.qza \
  --output-path $Data_aroma/table-filtered-sourdough_only

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Exported Data/classified/table-filtered-sourdough_only.qza as BIOMV210DirFmt to directory Data/aroma/table-filtered-sourdough_only


In [6]:
table = biom.load_table('Data/aroma/table-filtered-sourdough_only/feature-table.biom')

table_filtered_sourdough = table.to_dataframe()

table_filtered_sourdough = table_filtered_sourdough.T

print(f"Shape: {table_filtered_sourdough.shape}")
print(f"First 5 samples:\n{table_filtered_sourdough.index[:5]}")
print(f"First 5 features:\n{table_filtered_sourdough.columns[:5]}")

Shape: (125, 1021)
First 5 samples:
Index(['366291_386-LP4-ITS-0386', '366291_387-LP4-ITS-0387',
       '366291_388-LP4-ITS-0388', '366291_389-LP4-ITS-0389',
       '366291_390-LP4-ITS-0390'],
      dtype='object')
First 5 features:
Index(['5a460fcb277abe4f17393718edafa752', '8ba24a98c1b3b3b26befca415d3e3b47',
       '2543f41e7c955c359f41e8cb6fed4cbb', '7730deae77544493fe929a915bd690b1',
       '7037e38175f71e5339f866ac5e19ff9f'],
      dtype='object')


Prepare data

In [9]:
metadata = pd.read_csv("Data/raw/merged_output_usable.tsv", sep="\t") #using merged_output_usable because the spaces already substitued with _

aroma_columns = metadata.columns[-222:]

print(aroma_columns)

#so last 222 columns are aroma associated

Index(['ALCOHOLIC_D7', 'ANIMAL_FEED_D7', 'ANIMAL_STABLE_D7', 'APPLE_D7',
       'BANANA_D7', 'BEER_D7', 'BERRIES_D7', 'BREAD_D7', 'BUTTER_MILK_D7',
       'BUTYRIC_ACID_D7',
       ...
       'animal_score_D28', 'chemical_score_D28', 'body_odour_score_D28',
       'fruity_score_D28', 'maillard_score_D28', 'sour_score_D28',
       'ocean_score_D28', 'earthy_score_D28', 'fermented_dairy_score_D28',
       'nutty_score_D28'],
      dtype='object', length=222)


In [10]:
print("Now aroma df:", aroma_columns.shape, "Now metadata df:", metadata.shape)

Now aroma df: (222,) Now metadata df: (550, 304)


In [11]:
metadata_sd = metadata[metadata['sample_type'] == 'sourdough'].copy()

In [12]:
metadata_sd.head()
print("Now:", metadata_sd.shape)

Now: (125, 304)


In [13]:
# Check for missing values in aroma data
missing_aromas = metadata_sd[aroma_columns].isnull().sum()
if missing_aromas.any():
    print(f"\n⚠ Warning: Missing aroma values detected:")
    print(missing_aromas[missing_aromas > 0])


⚠ Warning: Missing aroma values detected:
PORRIDGE_D28                 21
WHOLE_GRAIN_D28              21
HAY_D28                      21
BREAD_D28                    21
CORN_D28                     21
                             ..
sour_score_D28               21
ocean_score_D28              21
earthy_score_D28             21
fermented_dairy_score_D28    21
nutty_score_D28              21
Length: 87, dtype: int64


In [14]:
metadata_sd[aroma_columns].isnull().sum(axis=1)

0       0
1       0
2       0
3       0
4      87
       ..
129     0
130     0
131     0
132    87
133     0
Length: 125, dtype: int64

In [15]:
print("Before:", metadata_sd.shape)
metadata_sd = metadata_sd.dropna(subset=aroma_columns, how='all')
print("After:", metadata_sd.shape)

Before: (125, 304)
After: (125, 304)


<div style="background-color: aliceblue; padding: 10px;">

so there are no samples that miss all aroma analyses

# Spearman correlation on genus level
<div style="background-color: skyblue; padding: 10px;">


In [16]:
# Extract genus from taxonomy
taxonomy['Genus'] = taxonomy['Taxon'].str.extract(r'g__([^;]+)')
taxonomy['Genus'] = taxonomy['Genus'].fillna('Unassigned')

print(f"\n✓ Extracted {taxonomy['Genus'].nunique()} unique genera")


✓ Extracted 1542 unique genera


In [17]:
# 1) Make sure taxonomy index is feature IDs
if 'Feature ID' in taxonomy.columns:
    taxonomy = taxonomy.set_index('Feature ID')

# 2) Extract genus
taxonomy['Genus'] = taxonomy['Taxon'].str.extract(r'g__([^;]+)')
taxonomy['Genus'] = taxonomy['Genus'].fillna('Unassigned')


# 3) Map ASVs to genus
feature_to_genus = taxonomy['Genus'].to_dict()

feature_table_genus = table_filtered_sourdough.copy()
feature_table_genus.columns = [
    feature_to_genus.get(col, 'Unknown') for col in feature_table_genus.columns
]

# 4) Collapse to genus
genus_table = feature_table_genus.groupby(level=0, axis=1).sum()

print("✓ Collapsed to genus level:", genus_table.shape[1], "genera")


/tmp/ipykernel_1740/2546118294.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  genus_table = feature_table_genus.groupby(level=0, axis=1).sum()


✓ Collapsed to genus level: 238 genera


In [49]:
genus_rel = genus_table.div(genus_table.sum(axis=1), axis=0)
top_genera = genus_rel.mean().sort_values(ascending=False).head(10)
print("\nTop 10 genera:")
for genus, abundance in top_genera.items():
    print(f"  {genus}: {abundance*100:.2f}%")



Top 10 genera:
  Saccharomyces: 70.61%
  Alternaria: 14.70%
  Unassigned: 5.44%
  Pyrenophora: 1.77%
  Pichia: 1.39%
  Aureobasidium: 1.22%
  Cladosporium: 1.08%
  Stemphylium: 0.44%
  Parastagonospora: 0.40%
  Sporobolomyces: 0.29%


Merging metadata & genus

In [19]:
metadata = metadata.set_index('sample ID')

In [20]:
# Merge genus data with metadata
merged_data = genus_table.join(metadata, how='inner')

print(f"✓ Merged data: {merged_data.shape}")

# Define fungal columns (from genus table)
fungal_columns = genus_table.columns.tolist()

print(f"\nFinal dataset:")
print(f"  Total samples: {len(merged_data)}")
print(f"  Fungal genera: {len(fungal_columns)}")
print(f"  Aroma attributes: {len(aroma_columns)}")
print(f"  Timepoints: {sorted(merged_data['day'].unique())}")
print(f"  Backgrounds: {merged_data['background'].unique().tolist()}")

✓ Merged data: (125, 541)

Final dataset:
  Total samples: 125
  Fungal genera: 238
  Aroma attributes: 222
  Timepoints: [7.0, 14.0, 21.0]
  Backgrounds: ['non-sterile', 'sterile']


In [21]:
fungal_columns = list(fungal_columns)
aroma_columns = list(aroma_columns)

complete_cases = merged_data[fungal_columns + aroma_columns].notna().all(axis=1).sum()
print(f"  Complete cases (no missing data): {complete_cases}/{len(merged_data)}")


  Complete cases (no missing data): 104/125


In [22]:
def calculate_correlations(data, fungal_columns, aroma_columns, min_samples=10):
    """Calculate Spearman correlations between fungi and aromas"""
    
    results = []
    
    for fungus in fungal_columns:
        for aroma in aroma_columns:
            # Remove samples where fungus is absent OR aroma is missing
            mask = (data[fungus] > 0) & (data[aroma].notna())
            n_samples = mask.sum()
            
            if n_samples < min_samples:
                continue
            
            x = data.loc[mask, fungus]
            y = data.loc[mask, aroma]
            
            # Skip if either variable is constant
            if x.nunique() < 2 or y.nunique() < 2:
                continue
            
            # Calculate Spearman correlation
            rho, p_value = spearmanr(x, y)
            
            results.append({
                'Fungus': fungus,
                'Aroma': aroma,
                'Spearman_rho': rho,
                'P_value': p_value,
                'N_samples': n_samples
            })
    
    results_df = pd.DataFrame(results)
    
    # Multiple testing correction (FDR)
    if len(results_df) > 0:
        results_df['FDR'] = multipletests(results_df['P_value'], method='fdr_bh')[1]
    
    return results_df


In [23]:
# Calculate correlations
cor_results = calculate_correlations(merged_data, fungal_columns, aroma_columns)

print(f"✓ Calculated {len(cor_results)} correlations")

✓ Calculated 9684 correlations


In [45]:
rho_threshold = 0.3
fdr_threshold = 0.1
sig_cors = cor_results[
    (cor_results['FDR'] < fdr_threshold) & 
    (cor_results['Spearman_rho'].abs() > rho_threshold)
].sort_values('Spearman_rho', key=abs, ascending=False)

print(f"✓ Significant correlations (FDR<{fdr_threshold}, |ρ|>{rho_threshold}): {len(sig_cors)}")

if len(sig_cors) > 0:
    print(f"\nTop 10 strongest correlations:")
    display_cols = ['Fungus', 'Aroma', 'Spearman_rho', 'FDR', 'N_samples']
    print(sig_cors.head(10)[display_cols].to_string(index=False))
    
    # Save results
    sig_cors.to_csv('significant_correlations.csv', index=False)
    print(f"\n✓ Saved to: significant_correlations.csv")
else:
    print("\n⚠ No significant correlations found!")
    print("  Consider lowering thresholds or checking data quality")

✓ Significant correlations (FDR<0.1, |ρ|>0.3): 0

⚠ No significant correlations found!
  Consider lowering thresholds or checking data quality


In [25]:
cor_results[['Spearman_rho', 'P_value', 'FDR']].describe()

,Spearman_rho,P_value,FDR
count,9684.000000,9684.000000,9684.000000
mean,0.008570,0.461018,0.875390
std,0.177406,0.288921,0.072596
min,-0.717127,0.000178,0.793966
25%,-0.096576,0.200898,0.793966
50%,0.015846,0.431062,0.861939
75%,0.112836,0.712428,0.949057
max,0.736213,1.000000,1.000000


In [61]:
cor_results.sort_values("Spearman_rho", key=abs, ascending=False).head(20)

,Fungus,Aroma,Spearman_rho,P_value,N_samples,FDR
8700,Tricellula,fruity_score_D28,0.736213,0.006330,12,0.793966
4645,Microstromatales_gen_Incertae_sedis,dayx_leavening_D28,-0.717127,0.005798,13,0.793966
6615,Pseudopithomyces,fruity_score_D28,-0.711918,0.006339,13,0.793966
8108,Starmerella,LACTIC_ACID_D28,0.709327,0.021603,10,0.793966
1251,Candida,BEER_D14,0.701793,0.010962,12,0.793966
1905,Curvibasidium,PINEAPPLE_D28,-0.694677,0.008410,13,0.793966
1903,Curvibasidium,BERRIES_D28,-0.694677,0.008410,13,0.793966
6608,Pseudopithomyces,dayx_pH_D28,-0.662900,0.013532,13,0.793966
8667,Tricellula,PINEAPPLE_D28,0.655273,0.020720,12,0.793966
6639,Pseudozyma,VINEGAR_D7,0.652649,0.015602,13,0.793966


<div style="background-color: aliceblue; padding: 10px;">

correlations are statistically weak also due to high number of testing

# Re-run Spearman correlation on family basis
<div style="background-color: skyblue; padding: 10px;">

to reduce reduction of statistical results through multiple testing issue

Reload taxonomy to avoid issues

In [26]:
taxonomy = pd.read_csv("Data/aroma/taxonomy.tsv", sep="\t")

In [27]:
# Extract genus from taxonomy
taxonomy['Family'] = taxonomy['Taxon'].str.extract(r'f__([^;]+)')
taxonomy['Family'] = taxonomy['Family'].fillna('Unassigned')

print(f"\n✓ Extracted {taxonomy['Family'].nunique()} unique families")


✓ Extracted 523 unique families


In [52]:
# 1) Make sure taxonomy index is feature IDs
if 'Feature ID' in taxonomy.columns:
    taxonomy = taxonomy.set_index('Feature ID')

# 2) Extract genus
taxonomy['Family'] = taxonomy['Taxon'].str.extract(r'f__([^;]+)')
taxonomy['Family'] = taxonomy['Family'].fillna('Unassigned')


# 3) Map ASVs to genus
feature_to_family = taxonomy['Family'].to_dict()

feature_table_family = table_filtered_sourdough.copy()
feature_table_family.columns = [
    feature_to_family.get(col, 'Unknown') for col in feature_table_family.columns
]

# 4) Collapse to genus
family_table = feature_table_family.groupby(level=0, axis=1).sum()

print("✓ Collapsed to family level:", family_table.shape[1], "families")


/tmp/ipykernel_1740/1221501352.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  family_table = feature_table_family.groupby(level=0, axis=1).sum()


✓ Collapsed to family level: 128 families


In [53]:
family_rel = family_table.div(family_table.sum(axis=1), axis=0)
top_family = family_rel.mean().sort_values(ascending=False).head(10)
print("\nTop 10 families:")
for family, abundance in top_family.items():
    print(f"  {family}: {abundance*100:.2f}%")



Top 10 families:
  Saccharomycetaceae: 70.61%
  Pleosporaceae: 17.10%
  Didymellaceae: 5.56%
  Pichiaceae: 1.39%
  Saccotheciaceae: 1.25%
  Cladosporiaceae: 1.08%
  Phaeosphaeriaceae: 0.45%
  Sporidiobolaceae: 0.31%
  Aspergillaceae: 0.29%
  Filobasidiaceae: 0.27%


In [54]:
# Merge genus data with metadata
merged_data_family = family_table.join(metadata, how='inner')

print(f"✓ Merged data: {merged_data_family.shape}")

# Define fungal columns (from genus table)
fungal_columns_family = family_table.columns.tolist()

print(f"\nFinal dataset:")
print(f"  Total samples: {len(merged_data_family)}")
print(f"  Fungal families: {len(fungal_columns_family)}")
print(f"  Aroma attributes: {len(aroma_columns)}")
print(f"  Timepoints: {sorted(merged_data['day'].unique())}")
print(f"  Backgrounds: {merged_data['background'].unique().tolist()}")

✓ Merged data: (125, 431)

Final dataset:
  Total samples: 125
  Fungal families: 128
  Aroma attributes: 222
  Timepoints: [7.0, 14.0, 21.0]
  Backgrounds: ['non-sterile', 'sterile']


In [55]:
fungal_columns_family = list(fungal_columns_family)
aroma_columns_family = list(aroma_columns)

complete_cases = merged_data_family[fungal_columns_family + aroma_columns].notna().all(axis=1).sum()
print(f"  Complete cases (no missing data): {complete_cases}/{len(merged_data_family)}")

  Complete cases (no missing data): 104/125


In [56]:
def calculate_correlations(data, fungal_columns_family, aroma_columns, min_samples=10):
    """Calculate Spearman correlations between fungi and aromas"""
    
    results = []
    
    for fungus in fungal_columns_family:
        for aroma in aroma_columns:
            # Remove samples where fungus is absent OR aroma is missing
            mask = (data[fungus] > 0) & (data[aroma].notna())
            n_samples = mask.sum()
            
            if n_samples < min_samples:
                continue
            
            x = data.loc[mask, fungus]
            y = data.loc[mask, aroma]
            
            # Skip if either variable is constant
            if x.nunique() < 2 or y.nunique() < 2:
                continue
            
            # Calculate Spearman correlation
            rho, p_value = spearmanr(x, y)
            
            results.append({
                'Fungus': fungus,
                'Aroma': aroma,
                'Spearman_rho': rho,
                'P_value': p_value,
                'N_samples': n_samples
            })
    
    results_df = pd.DataFrame(results)
    
    # Multiple testing correction (FDR)
    if len(results_df) > 0:
        results_df['FDR'] = multipletests(results_df['P_value'], method='fdr_bh')[1]
    
    return results_df


In [57]:
# Calculate correlations
cor_results_family = calculate_correlations(merged_data_family, fungal_columns_family, aroma_columns)

print(f"✓ Calculated {len(cor_results_family)} correlations")

✓ Calculated 7073 correlations


In [58]:
rho_threshold = 0.3
fdr_threshold = 0.1
sig_cors_family = cor_results_family[
    (cor_results_family['FDR'] < fdr_threshold) & 
    (cor_results_family['Spearman_rho'].abs() > rho_threshold)
].sort_values('Spearman_rho', key=abs, ascending=False)

print(f"✓ Significant correlations (FDR<{fdr_threshold}, |ρ|>{rho_threshold}): {len(sig_cors_family)}")

if len(sig_cors_family) > 0:
    print(f"\nTop 10 strongest correlations:")
    display_cols = ['Fungus', 'Aroma', 'Spearman_rho', 'FDR', 'N_samples']
    print(sig_cors_family.head(10)[display_cols].to_string(index=False))
    
    # Save results
    sig_cors_family.to_csv('significant_correlations.csv', index=False)
    print(f"\n✓ Saved to: significant_correlations.csv")
else:
    print("\n⚠ No significant correlations found!")
    print("  Consider lowering thresholds or checking data quality")

✓ Significant correlations (FDR<0.1, |ρ|>0.3): 0

⚠ No significant correlations found!
  Consider lowering thresholds or checking data quality


In [59]:
cor_results_family[['Spearman_rho', 'P_value', 'FDR']].describe()

,Spearman_rho,P_value,FDR
count,7073.000000,7073.000000,7073.000000
mean,0.006036,0.455335,0.862144
std,0.183923,0.288198,0.076272
min,-0.757588,0.000268,0.779112
25%,-0.102680,0.197269,0.779112
50%,0.012597,0.420228,0.840338
75%,0.112679,0.697379,0.929794
max,0.716981,1.000000,1.000000


In [60]:
cor_results_family.sort_values("Spearman_rho", key=abs, ascending=False).head(20)

,Fungus,Aroma,Spearman_rho,P_value,N_samples,FDR
3038,Metschnikowiaceae,dayx_leavening_D28,-0.757588,0.001698,14,0.779112
2354,Erysiphaceae,VINEGAR_D14,-0.734013,0.010119,11,0.779112
4277,Pezizomycotina_fam_Incertae_sedis,fruity_score_D28,0.716981,0.005812,13,0.779112
6593,Trichomonascaceae,LEAVES_D28,0.703856,0.003406,15,0.779112
6641,Trichomonascaceae,earthy_score_D28,0.703856,0.003406,15,0.779112
6614,Trichomonascaceae,MOLDY_D28,0.703856,0.003406,15,0.779112
6613,Trichomonascaceae,SOIL_D28,0.703856,0.003406,15,0.779112
3133,Microbotryomycetes_fam_Incertae_sedis,PINEAPPLE_D28,-0.694677,0.008410,13,0.779112
3131,Microbotryomycetes_fam_Incertae_sedis,BERRIES_D28,-0.694677,0.008410,13,0.779112
6574,Trichomonascaceae,LACTIC_ACID_D21,0.688973,0.001565,18,0.779112


### Continuing with sPLS

Prepare data to continue on R

In [78]:
metadata_sd = metadata_sd.set_index('sample ID')  # use your actual column name

In [86]:
family_rel.to_csv("Data/aroma/Analysis/X_fungi_family_rel.csv")             
metadata_sd[aroma_columns].to_csv("Data/aroma/Analysis/Y_aromas.csv")        
metadata_sd[['background']].to_csv("Data/aroma/Analysis/meta_background.csv")


This code was run on RStudio to create the circos plot